In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset= pd.read_csv('coughdf.csv')

In [3]:
dataset

,Unnamed: 0,datetime,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status,expert_labels_3,expert_labels_2,expert_labels_1
0,0,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,NaN,NaN,NaN
1,1,2020-04-13T21:30:59.801831+00:00,0.9609,31.3,34.8,15.0,male,False,False,healthy,NaN,NaN,NaN
2,2,2020-04-12T04:02:18.159383+00:00,0.9301,40.0,-75.1,34.0,male,True,False,healthy,NaN,NaN,NaN
3,3,2020-04-15T01:03:59.029326+00:00,0.0482,-16.5,-71.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020-04-13T22:23:06.997578+00:00,0.9968,NaN,NaN,21.0,male,False,False,healthy,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19986,19986,2020-06-05T12:21:20.437869+00:00,0.9866,NaN,NaN,40.0,female,False,False,healthy,NaN,NaN,NaN
19987,19987,2020-05-03T05:44:10.313306+00:00,0.9921,3.4,-76.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19988,19988,2020-04-10T16:12:50.441368+00:00,0.6514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19989,19989,2020-04-13T15:31:11.218368+00:00,0.5634,41.9,60.2,NaN,female,False,True,COVID-19,NaN,NaN,NaN


In [4]:
updt=dataset.drop('expert_labels_3', inplace=True, axis=1)
updt=dataset.drop('expert_labels_2', inplace=True, axis=1)
updt=dataset.drop('expert_labels_1', inplace=True, axis=1)
updt=dataset.drop('datetime', inplace=True, axis=1)
updt=dataset.dropna(subset=['status'])
updt=dataset.dropna(subset=['gender','respiratory_condition','fever_muscle_pain'],how='all')
updt=updt.replace(to_replace =["other"], 
                            value ="female")
updt=updt.replace(to_replace =["symptomatic"], 
                            value ="COVID-19")


In [5]:
updt


,Unnamed: 0,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status
0,0,0.9609,31.3,34.8,15.0,male,False,False,healthy
1,1,0.9609,31.3,34.8,15.0,male,False,False,healthy
2,2,0.9301,40.0,-75.1,34.0,male,True,False,healthy
4,4,0.9968,NaN,NaN,21.0,male,False,False,healthy
6,6,0.0306,13.8,-89.6,NaN,female,False,True,COVID-19
...,...,...,...,...,...,...,...,...,...
19983,19983,0.9820,19.0,72.8,32.0,male,False,False,healthy
19985,19985,0.6354,41.0,28.7,35.0,female,False,False,healthy
19986,19986,0.9866,NaN,NaN,40.0,female,False,False,healthy
19989,19989,0.5634,41.9,60.2,NaN,female,False,True,COVID-19


In [6]:
from sklearn import preprocessing
levelencoder= preprocessing.LabelEncoder()
updt['fever_muscle_pain']= levelencoder.fit_transform(updt['fever_muscle_pain'])
updt['gender']= levelencoder.fit_transform(updt['gender'])
updt['respiratory_condition']= levelencoder.fit_transform(updt['respiratory_condition'])
updt['status']= levelencoder.fit_transform(updt['status'])


In [7]:
updt

,Unnamed: 0,cough_detected,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,status
0,0,0.9609,31.3,34.8,15.0,1,0,0,1
1,1,0.9609,31.3,34.8,15.0,1,0,0,1
2,2,0.9301,40.0,-75.1,34.0,1,1,0,1
4,4,0.9968,NaN,NaN,21.0,1,0,0,1
6,6,0.0306,13.8,-89.6,NaN,0,0,1,0
...,...,...,...,...,...,...,...,...,...
19983,19983,0.9820,19.0,72.8,32.0,1,0,0,1
19985,19985,0.6354,41.0,28.7,35.0,0,0,0,1
19986,19986,0.9866,NaN,NaN,40.0,0,0,0,1
19989,19989,0.5634,41.9,60.2,NaN,0,0,1,0


training CNN


In [8]:
m = updt.iloc[:, 1:].values

In [9]:
m

array([[  0.9609,  31.3   ,  34.8   , ...,   0.    ,   0.    ,   1.    ],
       [  0.9609,  31.3   ,  34.8   , ...,   0.    ,   0.    ,   1.    ],
       [  0.9301,  40.    , -75.1   , ...,   1.    ,   0.    ,   1.    ],
       ...,
       [  0.9866,      nan,      nan, ...,   0.    ,   0.    ,   1.    ],
       [  0.5634,  41.9   ,  60.2   , ...,   0.    ,   1.    ,   0.    ],
       [  0.9954,      nan,      nan, ...,   1.    ,   0.    ,   1.    ]])

In [10]:
m.shape

(11259, 8)

In [11]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(m[:, 1:4])
m[:, 1:4] = imputer.transform(m[:, 1:4])

In [12]:
def split_sequences(sequences, n_steps):
	a, b = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
    
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		a.append(seq_x)
		b.append(seq_y)
	return array(a), array(b)

In [13]:
from numpy import array
a, b = split_sequences(m, 8)

In [14]:
a

array([[[ 9.60900000e-01,  3.13000000e+01,  3.48000000e+01, ...,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.60900000e-01,  3.13000000e+01,  3.48000000e+01, ...,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.30100000e-01,  4.00000000e+01, -7.51000000e+01, ...,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00],
        ...,
        [ 8.93700000e-01,  3.06975696e+01,  6.76545883e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.88300000e-01,  3.94000000e+01,  6.72000000e+01, ...,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.45600000e-01,  3.06975696e+01,  6.76545883e+00, ...,
          1.00000000e+00,  1.00000000e+00,  0.00000000e+00]],

       [[ 9.60900000e-01,  3.13000000e+01,  3.48000000e+01, ...,
          1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 9.30100000e-01,  4.00000000e+01, -7.51000000e+01, ...,
          1.00000000e+00,  1.00000000e

In [15]:
a.shape

(11252, 8, 7)

In [16]:
b

array([0., 1., 0., ..., 1., 0., 1.])

In [17]:
from sklearn.model_selection import train_test_split
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size = 0.2, random_state = 1)

In [18]:
n_features = a.shape[2]

In [19]:
n_features

7

In [20]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [ ]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(8, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
#model.add(Dense(1))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='mse',metrics = ['accuracy'])
# fit model
#model.fit(a, b, epochs=10, verbose=0)
model.fit(a_train,  b_train,  epochs=1000, validation_data=(a_test,b_test))

Epoch 1/1000
282/282 [==============================] - 1s 2ms/step - loss: 0.2466 - accuracy: 0.7520 - val_loss: 0.2474 - val_accuracy: 0.7526
Epoch 2/1000
282/282 [==============================] - 0s 2ms/step - loss: 0.2424 - accuracy: 0.7576 - val_loss: 0.2474 - val_accuracy: 0.7526
Epoch 3/1000
282/282 [==============================] - 0s 1ms/step - loss: 0.2424 - accuracy: 0.7576 - val_loss: 0.2474 - val_accuracy: 0.7526
Epoch 4/1000
282/282 [==============================] - 0s 2ms/step - loss: 0.2424 - accuracy: 0.7576 - val_loss: 0.2474 - val_accuracy: 0.7526
Epoch 5/1000
282/282 [==============================] - 0s 1ms/step - loss: 0.2424 - accuracy: 0.7576 - val_loss: 0.2474 - val_accuracy: 0.7526
Epoch 6/1000
282/282 [==============================] - 0s 1ms/step - loss: 0.2424 - accuracy: 0.7576 - val_loss: 0.2474 - val_accuracy: 0.7526
Epoch 7/1000
282/282 [==============================] - 0s 2ms/step - loss: 0.2424 - accuracy: 0.7576 - val_loss: 0.2474 - val_accuracy:

In [34]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 7, 64)             960       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                9650      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 51        
Total params: 10,661
Trainable params: 10,661
Non-trainable params: 0
_________________________________________________________________


In [35]:
#model.fit(a_train, b_train, batch_size=128, epochs=1000, validation_data=(a_test, b_test), verbose=0)


In [36]:
test_loss, test_acc = model.evaluate(a_test,b_test)
print('test_acc: ',test_acc)

71/71 [==============================] - 0s 851us/step - loss: 0.2474 - accuracy: 0.7526
test_acc:  0.7525544166564941
